In [1]:
import gzip
import argparse
import numpy as np
import paddle.v2 as paddle
from CSRNet import *
from reader import data_reader

In [2]:
class Struct():
    def __init__(self):
        pass
args = Struct()
args.model_gz_path='/share/dong/Result/Paddle_Vgg16.tar.gz'
args.train_json_path = './baidu_train.json'
args.test_json_path = './baidu_test.json'
args.model_output_prefix='/share/dong/Result/CSR'
args.BATCH_SIZE = 1
args.num_passes = 50
def train():
    paddle.init(use_gpu=True, trainer_count=1)
    model = CSRNet()
    params = paddle.parameters.create(model.cost)
    logger.info("load model parameters from %s" % args.model_gz_path)
    params.init_from_tar(gzip.open(args.model_gz_path, 'r'))
#     optimizer = paddle.optimizer.Momentum(
#         momentum=0.95,
# #         regularization=paddle.optimizer.L2Regularization(rate=0.0005 * args.BATCH_SIZE),
#         learning_rate=0.1,
#         learning_rate_decay_a=0.1,
#         learning_rate_decay_b=128000 * 35,
#         learning_rate_schedule="discexp" )
    optimizer = paddle.optimizer.Momentum(
        momentum=0.95,
        learning_rate=1e-7)
    trainer = paddle.trainer.SGD(
        cost=model.cost,
        parameters=params,
        update_equation=optimizer) 
    def __event_handler__(event):
        if isinstance(event, paddle.event.EndIteration):
            num_samples = event.batch_id * args.BATCH_SIZE
            if event.batch_id % 100 == 0:
                logger.warning("Pass %d, Samples %d, Cost %f, %s" % (
                    event.pass_id, num_samples, event.cost, event.metrics))

            if event.batch_id % 1000 == 0:
                feeding_index={"image":0,'label':1}
                result = trainer.test(
                    reader=paddle.batch(
                        data_reader(args.test_json_path),
                        batch_size=args.BATCH_SIZE),
                    feeding=feeding_index)
                logger.warning("Test %d-%d, Cost %f, %s" %
                               (event.pass_id, event.batch_id, result.cost,
                                result.metrics))

                path = "{}-pass-{}-batch-{}-test-{}.tar.gz".format(
                    args.model_output_prefix, event.pass_id, event.batch_id,
                    result.cost)
                with gzip.open(path, 'w') as f:
                    trainer.save_parameter_to_tar(f)
    feeding_index={"image":0,'label':1}
    trainer.train(
        reader=paddle.batch(
            paddle.reader.shuffle(
                data_reader(args.train_json_path), buf_size=args.BATCH_SIZE*2),
            batch_size=args.BATCH_SIZE),
        feeding=feeding_index,
        event_handler=__event_handler__,
        num_passes=args.num_passes)

In [ ]:
train()

[INFO 2018-08-08 01:01:34,706 layers.py:2716] output for __conv_0__: c = 64, h = 540, w = 960, size = 33177600
[INFO 2018-08-08 01:01:34,709 layers.py:3361] output for __batch_norm_0__: c = 64, h = 540, w = 960, size = 33177600
[INFO 2018-08-08 01:01:34,711 layers.py:2716] output for __conv_1__: c = 64, h = 540, w = 960, size = 33177600
[INFO 2018-08-08 01:01:34,712 layers.py:3361] output for __batch_norm_1__: c = 64, h = 540, w = 960, size = 33177600
[INFO 2018-08-08 01:01:34,714 layers.py:2858] output for __pool_0__: c = 64, h = 270, w = 480, size = 8294400
[INFO 2018-08-08 01:01:34,716 layers.py:2716] output for __conv_2__: c = 128, h = 270, w = 480, size = 16588800
[INFO 2018-08-08 01:01:34,718 layers.py:3361] output for __batch_norm_2__: c = 128, h = 270, w = 480, size = 16588800
[INFO 2018-08-08 01:01:34,720 layers.py:2716] output for __conv_3__: c = 128, h = 270, w = 480, size = 16588800
[INFO 2018-08-08 01:01:34,722 layers.py:3361] output for __batch_norm_3__: c = 128, h = 270,

[WARNING 2018-08-08 01:44:09,314 <ipython-input-2-40550b22479b>:47] Test 1-1000, Cost 20.586802, {}
[WARNING 2018-08-08 01:44:55,925 <ipython-input-2-40550b22479b>:36] Pass 1, Samples 1100, Cost 0.016788, {}
[WARNING 2018-08-08 01:46:51,510 <ipython-input-2-40550b22479b>:36] Pass 1, Samples 1200, Cost 0.021624, {}
[WARNING 2018-08-08 01:47:45,799 <ipython-input-2-40550b22479b>:36] Pass 1, Samples 1300, Cost 0.000000, {}
[WARNING 2018-08-08 01:49:56,179 <ipython-input-2-40550b22479b>:36] Pass 1, Samples 1400, Cost 44.561752, {}
[WARNING 2018-08-08 01:52:09,985 <ipython-input-2-40550b22479b>:36] Pass 1, Samples 1500, Cost 1.035713, {}
[WARNING 2018-08-08 01:53:02,839 <ipython-input-2-40550b22479b>:36] Pass 1, Samples 1600, Cost 0.068959, {}
[WARNING 2018-08-08 01:55:16,457 <ipython-input-2-40550b22479b>:36] Pass 1, Samples 1700, Cost 0.195076, {}
[WARNING 2018-08-08 01:56:08,271 <ipython-input-2-40550b22479b>:36] Pass 1, Samples 1800, Cost 0.003320, {}
[WARNING 2018-08-08 01:58:23,771 <i